In [2]:
from dataclasses import dataclass
from scipy.optimize import least_squares
from scipy.stats import nbinom
import pandas as pd
import numpy as np

In [8]:
df = pd.read_table("histogram.tsv", names=("depth", "count")).iloc[:, :2000]

In [9]:
df

,depth,count
0,6,159858055
1,7,106647935
2,8,72263818
3,9,49778791
4,10,34925176
...,...,...
177614,69088931,1
177615,118201993,1
177616,118289506,1
177617,166067752,1


In [77]:
@dataclass
class KmerProfileModel:
    depths: np.ndarray
    counts: np.ndarray
    peaks: int = 4

    def get_peak_counts(self, r, p, fraction):
        total_count = self.counts.sum()
        log_likelihood = nbinom.logpmf(self.depths, r, p)
        return np.exp(np.log(total_count * fraction) + log_likelihood)

    def get_model_counts(self, base_depth, bias, fractions):
        total_count = self.counts.sum()
        model_counts = np.zeros(self.counts.shape)
        for i in range(self.peaks):
            r = (i+1) * base_depth / bias
            mu = (i+1) * base_depth
            p = r / (r + mu)
            fraction = fractions[i]
            model_counts += self.get_peak_counts(r, p, fraction)
        return model_counts

    def get_residual_sum_of_squares(self, base_depth, bias, fractions):
        model_counts = self.get_model_counts(base_depth, bias, fractions)
        return ((self.counts - model_counts) ** 2).sum()

    
    def get_cost(self, params, *args, **kw):
        base_depth = params[0]
        bias = params[1]
        fractions = params[2:]
        return self.get_residual_sum_of_squares(base_depth, bias, fractions)

    def optimize(self, estimated_base_depth, *args, **kw):
        initial_bias = 0.5
        initial_fractions = np.ones(self.peaks) / (self.peaks + 0.01)
        initial_params = [estimated_base_depth, initial_bias, *initial_fractions]
        lower_bounds = [5, 0.001] + [1e-10] * self.peaks
        upper_bounds = [1000, 1000, ] + [1 - 1e-10] * self.peaks
        bounds = (lower_bounds, upper_bounds)
        return least_squares(self.get_cost, initial_params, bounds=bounds, *args, **kw)





In [78]:
model = KmerProfileModel(df['depth'], df['count'], 1)

In [79]:
model.optimize(220)

     message: The maximum number of function evaluations is exceeded.
     success: False
      status: 0
         fun: [ 1.156e+16]
           x: [ 5.015e+00  5.575e-01  6.910e-01]
        cost: 6.685806413440147e+31
         jac: [[ 4.697e+14 -2.438e+12  5.486e+13]]
        grad: [ 5.431e+30 -2.819e+28  6.344e+29]
  optimality: 2.817236357810717e+31
 active_mask: [0 0 0]
        nfev: 300
        njev: 230

In [39]:
model.get_peak_counts(200, 0.9, 1)

array([124128.26629807, 365291.7551057 , 945192.41633604, ...,
            0.        ,      0.        ,      0.        ])

In [25]:
nbinom.pmf([10, 100, 200], 100, 0.5)

array([3.28442046e-20, 2.81742395e-02, 6.80441503e-10])

In [26]:
nbinom.logpmf([10, 100, 200], 100, 0.5)

array([-44.86251164,  -3.56934721, -21.10827926])

In [27]:
np.exp(-44.86251164)

3.2844204718223304e-20

In [ ]:
def get_log_likelihood()